# German Credit Analysis, Using DNN

# Summary of the project

This purpose of this project is to replicate the findings in "Combining Feature Selection and Neural Networks for Solving Classification Problems," O'Dea, P., Griffith, J., O'Riordan, C. This paper used the German Credit data set and combined feature selection and Neural Networks to solve classification problems.

This approach took the form of two phases. The first phase was to use Information Theory to determine the attributes of the German credit data set that are the most important in classifiying the data record as good credit or bad credit.

# Introduction to Data Mining Methods and their Applications

Data mining is deployed in many different "classification and association rules, and to item-set recognition and sequential pattern recoginition problems" (O'Dea, p. 1). Many of these methods are outlined in Provost & Fawcett pages 20-23.

When deploying classification methods, the key objective is to predict which class an item belong to based on a set of attributes (Provost & Fawcett, p. 20). Applications of classification involve prediciting customer churn, or in classifying what class an item belongs to. Examples of methods involved in classifications are Neural Networks, score cards, and decision trees.

Regression, or "value estimation" (Provost & Fawcett, p. 21) is a technique that attempts to predict the numerical value based on past data. The technique uses a linear mathematical model (often using the least squares approach), that can predict the dependent variable from an independent variable. Applications of regression models include predicting the temperature on a given day based on previous days (or seasonal data).

Similarity matching used known data about items and attempts to identify similar items. Applications of similarity matching include finding customers that are similar to your best customer.

Clustering methods is an unsupervisied (no target attribute), that attempts to cluster or group like items together. Applications of clustering include finding segments in groups of customers.

Co-occurrence grouping is a data mining method that attempts to associate items based on their transactions. It is also called assiaction rule discovery or market-basket analysis. An application of co-occurrence grouping is to suggest products to customers based on the items they have in their shopping cart. A famous example of co-occurrence grouping is the diapers and beer association (Whitehorn, 2006).

Profiling is a data mining method that tries to describe the behavior of individuals. It is most useful in "fraud detection and monitoring intrusion detection" (Provost & Fawcett, p. 22).

Link prediction attempt to determine whether or not a link between items should exist based on associated other links. A common applicaiton of link prediction is friend sugestions in Facebook.

Data reduction is a data mining method that attempts to reduce the complexity and volume of a large data set to a more manageable size by pruning the less important information. Data reduction often results in data loss, but at the gain of performance and easier understanding of the data set involved.

Causal modeling tries to find links between events and actions.

# Techniques and Approaches

The techniques used in this model include data segmentation, data reduction, and classification. The first phase of the project is to determine the most important attributes to use when training and testing the model, and reduce the data set to those attributes. This is done using Information Theory.

The second phase is to process the reduced data set into a form that can be easily used by the model. This is done by binning the data sets for numerical data into clusters.

# Cross Industry Standard Process for Data Mining (CRISP-DM)

We need a process for data mining in order to have a reasonable amount of “consistency, repeatability, and objectiveness” (Provost, 2013, p. 27) of the outcomes. The CRISP-DM method is one of the most popular data mining processes used in the industry. The CRISP-DM process is an iterative process. Each iteration helps to inform about the data.


## Business Understanding

The business understanding defines the “problem to be solved” (p. 28). The problem here is a feature selection and classification problem: Using the German dataset we need to predict if an applicant is a “good or bad credit risk” (O’Dae, Griffith, O’Riordan; p. 6).





Some necessary libraries:

In [1]:
from __future__ import division
from math import log
import pandas as pd

## Data Understanding

The data is composed of twenty attributes. Using entropy and information gain, seven attributes were selected as the most useful in determining the credit worthiness of a candidate based on information gain. The attributes selected were: the status of existing checking account, the credit duration in months, credit history, credit amount, savings accounts/bonds, housing (rent, own, for free), and whether or not the person is a foreign worker.


In [2]:
german = pd.read_csv('MGMT635_GermanCreditData.csv')

In [3]:
german.head()

,status,duration,credit_history,purpose,credit_amount,savings,employment_duration,installment_rate,personal_status,debtors,...,property,age,installment_plans,housing,existing_credits,job,liable_people,telephone,foreign_worker,target
0,11,6,34,43,1169,65,75,4,93,101,...,121,67,143,152,2,173,1,192,201,1
1,12,48,32,43,5951,61,73,2,92,101,...,121,22,143,152,1,173,1,191,201,2
2,14,12,34,46,2096,61,74,2,93,101,...,121,49,143,152,1,172,2,191,201,1
3,11,42,32,42,7882,61,74,2,93,103,...,122,45,143,153,1,173,2,191,201,1
4,11,24,33,40,4870,61,73,3,93,101,...,124,53,143,153,2,173,2,191,201,2


There are two types of attributes: Categorical which are in dictionary 1, and numerical which is in dictionary 2.

In [4]:
attribute_dict = {1:['status', 'credit_history', 'purpose', 'savings',
                    'employment_duration', 'personal_status', 'debtors','property', 'installment_plans', 'housing', 
                    'job', 'telephone', 'foreign_worker'], 
                  2:['duration', 'credit_amount', 'installment_rate', 'residence', 'age', 'existing_credits',
                     'liable_people']}

A good way to get insights into the data set is to get some basic statistics. Below is a display of some basic statisitcal information which will tell us things such as mean, standard deviation, minimum and maximum values. For categorical data, the mode would be a useful statistic.

In [5]:
german.describe()

,status,duration,credit_history,purpose,credit_amount,savings,employment_duration,installment_rate,personal_status,debtors,...,property,age,installment_plans,housing,existing_credits,job,liable_people,telephone,foreign_worker,target
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,12.577000,20.903000,32.54500,47.148000,3271.258000,62.105000,73.384000,2.973000,92.68200,101.145000,...,122.358000,35.546000,142.675000,151.929000,1.407000,172.904000,1.155000,191.404000,201.037000,1.300000
std,1.257638,12.058814,1.08312,40.095333,2822.736876,1.580023,1.208306,1.118715,0.70808,0.477706,...,1.050209,11.375469,0.705601,0.531264,0.577654,0.653614,0.362086,0.490943,0.188856,0.458487
min,11.000000,4.000000,30.00000,40.000000,250.000000,61.000000,71.000000,1.000000,91.00000,101.000000,...,121.000000,19.000000,141.000000,151.000000,1.000000,171.000000,1.000000,191.000000,201.000000,1.000000
25%,11.000000,12.000000,32.00000,41.000000,1365.500000,61.000000,73.000000,2.000000,92.00000,101.000000,...,121.000000,27.000000,143.000000,152.000000,1.000000,173.000000,1.000000,191.000000,201.000000,1.000000
50%,12.000000,18.000000,32.00000,42.000000,2319.500000,61.000000,73.000000,3.000000,93.00000,101.000000,...,122.000000,33.000000,143.000000,152.000000,1.000000,173.000000,1.000000,191.000000,201.000000,1.000000
75%,14.000000,24.000000,34.00000,43.000000,3972.250000,63.000000,75.000000,4.000000,93.00000,101.000000,...,123.000000,42.000000,143.000000,152.000000,2.000000,173.000000,1.000000,192.000000,201.000000,2.000000
max,14.000000,72.000000,34.00000,410.000000,18424.000000,65.000000,75.000000,4.000000,94.00000,103.000000,...,124.000000,75.000000,143.000000,153.000000,4.000000,174.000000,2.000000,192.000000,202.000000,2.000000


## Data Preparation

O’Dea, et. al. used the thermometer coding scheme in order to represent the attribute values. For example for the status attribute, the values less-200DM were coded as {001}, over-200DM was coded as {011}, and no-account was coded as {111}. Also, continuous data, such as duration and credit_amount were binned (or bucketed)  in order to aggregate the data into a more useful form for the model to use. Table 2 of Dea, shows the binary representation used to represent the inputs to the neural network (25 inputs in all).

### Feature Selection
Using information theory, as outlined in Provost and Fawcett (2013). We calculated the information gained for each of the attributes. As an example we will display the calculations for the attribute `status`.

First, based on the data set, the overall probabilities of good credit or bad credit can be calculated:

In [6]:
p_parent_good = german.where(german.target==1).dropna().shape[0]/german.shape[0]
print("Probability of good credit = {}".format(p_parent_good))
p_parent_bad = german.where(german.target==2).dropna().shape[0]/german.shape[0]
print("Probability of bad credit = {}".format(p_parent_bad))

Probability of good credit = 0.7
Probability of bad credit = 0.3


The total entropy of the data set is calculated:

In [7]:
parent_entropy = - (p_parent_good * log(p_parent_good, 2) + p_parent_bad * log(p_parent_bad, 2))
print("Parent Entropy = {}".format(parent_entropy))

Parent Entropy = 0.881290899231


In order to understand how informative the attribute is we need to calculate the information gained. This is done by calculating how much the attribute reduces the entropy of the segmentations created by splitting the data set along the values of the attribute.

In [8]:
feature_values_for_status = [11,12,13,14]

status_value_series = {}
#Split the data set along the values of the attributes.
for value in feature_values_for_status:
    status_value_series[value] = german.where(german['status']==value).dropna().target

IG_children = 0
for key, series in status_value_series.iteritems():
    p_status_value = series.shape[0] / german.shape[0]
    p_series_good = series.where(series==1).dropna().shape[0] / series.shape[0]
    p_series_bad = series.where(series==2).dropna().shape[0] / series.shape[0]
    entropy_child = -(p_series_good * log(p_series_good, 2) + p_series_bad * log(p_series_bad, 2))
    IG_children = IG_children + (p_status_value * entropy_child)
    print("Probability of value {}: {}".format(key, p_status_value))
    print("Probability for value {}, to have good credit: {}".format(key, p_series_good))
    print("Probability for value {}, to have bad credit: {}".format(key, p_series_bad))
    print("Entropy of child with value {}: {}".format(key, entropy_child))
    print("--------------------------------------------------------")
    

Probability of value 11: 0.274
Probability for value 11, to have good credit: 0.507299270073
Probability for value 11, to have bad credit: 0.492700729927
Entropy of child with value 11: 0.999846262849
--------------------------------------------------------
Probability of value 12: 0.269
Probability for value 12, to have good credit: 0.609665427509
Probability for value 12, to have bad credit: 0.390334572491
Entropy of child with value 12: 0.965015120503
--------------------------------------------------------
Probability of value 13: 0.063
Probability for value 13, to have good credit: 0.777777777778
Probability for value 13, to have bad credit: 0.222222222222
Entropy of child with value 13: 0.764204506509
--------------------------------------------------------
Probability of value 14: 0.394
Probability for value 14, to have good credit: 0.883248730964
Probability for value 14, to have bad credit: 0.116751269036
Entropy of child with value 14: 0.519949823177
-------------------------

We use the sum of the products of the children entropies and probabilities and subtract it from the entropy of the parent:

In [9]:
IG = parent_entropy - IG_children
print("Information Gain for attribute status: {}".format(IG))

Information Gain for attribute status: 0.0947388415526


### Binning Attributes

It is useful to bin or basket together numerical data. You loose data doing this but the outcome is more useful for the neural network model.

In [10]:
german.duration = pd.cut(german.duration, bins=4, labels=False, include_lowest=True)
german.credit_amount = pd.cut(german.credit_amount, bins=4, labels=False, include_lowest=True)


##### Noted as most relevant attributes from the paper are:

In [11]:
german2 = german[['duration', 'credit_history', 'credit_amount', 'savings', 
                  'status', 'housing', 'foreign_worker', 'target']]

In [12]:
german2.head()


,duration,credit_history,credit_amount,savings,status,housing,foreign_worker,target
0,0,34,0,65,11,152,201,1
1,2,32,1,61,12,152,201,2
2,0,34,0,61,14,152,201,1
3,2,32,1,61,11,153,201,1
4,1,33,1,61,11,153,201,2


### Chop up the data (training, testing)

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_data = german2.drop('target',axis=1)

In [15]:
y_labels = german['target']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.2, random_state=101)

In [17]:
german.columns

Index([u'status', u'duration', u'credit_history', u'purpose', u'credit_amount',
       u'savings', u'employment_duration', u'installment_rate',
       u'personal_status', u'debtors', u'residence', u'property', u'age',
       u'installment_plans', u'housing', u'existing_credits', u'job',
       u'liable_people', u'telephone', u'foreign_worker', u'target'],
      dtype='object')

# Model Based on Neural Networks

## Model Selection

We chose to use a high level api provided by TensorFlow. TensorFlow has a number of estimator API’s, which allows a user to tap into pre-built models. We happened to choose the DNN Classifier (dense neural network). We also ran the data through TF’s Linear Classifier, but decided to go with the dense neural network because it produced better results. The DNN Classifier is also an ideal model for binary classification problems - in our case we needed to predict 1 or 2 (Good vs Bad).






In [18]:
import tensorflow as tf

### Assign feature columns for TensorFlow (categorical)

In [19]:
feature_columns = []
for key in german2.drop('target', axis=1).columns:
    feature_columns.append(tf.feature_column.numeric_column(key=key))

In [20]:
for attribute in attribute_dict.values():
    print(attribute)

['status', 'credit_history', 'purpose', 'savings', 'employment_duration', 'personal_status', 'debtors', 'property', 'installment_plans', 'housing', 'job', 'telephone', 'foreign_worker']
['duration', 'credit_amount', 'installment_rate', 'residence', 'age', 'existing_credits', 'liable_people']


## Model Implementation

First, the user would define the various features. The main part of defining the features in TF is setting the type of date, for instance is the data categorical or numeric. In our case all the categorical attributes were translated into numbers, so we set each feature to numeric.

Second, we defined the input function. This packages the data in the form of a Pandas dataframe and inserts it into the TensorFlow model. Within this input function you can set the number of epochs, batch size etc. 


In [21]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

Third, we defined the actual model. In this function the user would set the number of neurons as well as the number of hidden layers. The model also comes equipped with gradient descent optimizers - the default is the Adagrad Algorithm which is what we used.

In [22]:
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10],n_classes=4, feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f860158d3d0>, '_model_dir': '/tmp/tmp5wTR4y', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


Lastly, we ran the model by using the train function on the model. This function took in the argument of how many steps the model should run for and your input function. 

In [23]:
dnn_model.train(input_fn=input_func,steps=500)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp5wTR4y/model.ckpt.
INFO:tensorflow:loss = 18.705767, step = 1
INFO:tensorflow:global_step/sec: 299.016
INFO:tensorflow:loss = 6.2550282, step = 101 (0.338 sec)
INFO:tensorflow:global_step/sec: 491.524
INFO:tensorflow:loss = 5.184935, step = 201 (0.202 sec)
INFO:tensorflow:global_step/sec: 400.993
INFO:tensorflow:loss = 5.175823, step = 301 (0.250 sec)
INFO:tensorflow:global_step/sec: 408.634
INFO:tensorflow:loss = 5.267221, step = 401 (0.244 sec)
INFO:tens

### Evaluate the predictions

In [24]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,shuffle=False)

The tensorflow api comes equipped with a evaluation method in which you can input your testing set and compare against the actual targets. In addition to this, we created a scoring system where a penalty would be applied for wrong predictions. 5 points would be allotted if the model predicted a 1 but the actual was 2. 1 point was was allotted if it had predicted a 2 rather than the actual value 1. 0 points for correct answers. In short we looked for a model that maximized the accuracy percentage and minimized the penalty score.

A python script was developed to take in a range of parameters (ie. layers and neurons) and test the various models in bulk. We took the averages and graphed the various outputs using excel. 


In [25]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-02-16:26:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp5wTR4y/model.ckpt-500
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-02-16:26:56
INFO:tensorflow:Saving dict for global step 500: accuracy = 0.655, average_loss = 0.64779013, global_step = 500, loss = 6.4779015
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: /tmp/tmp5wTR4y/model.ckpt-500


{'accuracy': 0.655,
 'average_loss': 0.64779013,
 'global_step': 500,
 'loss': 6.4779015}

# References

Whitehorn, M. (2006). "The parable of the beer and diapers", The Register. Retrieved from: https://www.theregister.co.uk/2006/08/15/beer_diapers/

Provost, F. & Fawcett, T. (2013). "Data Science for Business". O'Reilly. Sebastopol, CA.

O'Dea, P., Griffith, J., O'Riordan, C. "Combining Feature Selection and Neural Networks for Solving Classification Problems". National University of Ireland. Galway, Galway.